In [4]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("local")
         .appName("read-postgres")
         # Add postgres jar
         .config("spark.driver.extraClassPath", "/home/jovyan/work/jars/postgresql-9.4.1207.jar")
         .getOrCreate())
sc = spark.sparkContext

22/01/25 02:38:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
postgres_db = "jdbc:postgresql://postgres/airflow"
postgres_user = "airflow"
postgres_pwd = "airflow"

In [6]:
dataSource = (
    spark.read
    .format("jdbc")
    .option("url", postgres_db)
    .option("dbtable", "public.stock")
    .option("user", postgres_user)
    .option("password", postgres_pwd)
    .load()
)

In [7]:
# IMPORT THƯ VIỆN
import pyspark
from pyspark.sql import functions as f
from pyspark.sql.functions import col

In [8]:
#dataSource = spark.read.options(header = 'True', inferSchema = 'True', delimiter = ',').csv("/Users/macbookquan/Desktop/HK 1 -4/BigData/dataset_final/raw.csv")
dataSource.printSchema()
dataSource.show(5)

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj_Close: double (nullable = true)
 |-- Volume: long (nullable = true)
 |-- Symbol: string (nullable = true)



+-------------------+------------------+------------------+------------------+------------------+------------------+-------+------+
|               Date|              Open|              High|               Low|             Close|         Adj_Close| Volume|Symbol|
+-------------------+------------------+------------------+------------------+------------------+------------------+-------+------+
|2021-01-25 00:00:00| 168.8800048828125|172.49000549316406|168.22999572753906|170.38999938964844|165.14793395996094|3353100|   MMM|
|2021-01-26 00:00:00|173.77999877929688|176.83999633789062| 172.0800018310547| 175.9499969482422|  170.536865234375|5148900|   MMM|
|2021-01-27 00:00:00|177.30999755859375|187.27000427246094|177.16000366210938|186.64999389648438|180.90768432617188|8097000|   MMM|
|2021-01-28 00:00:00|             185.0| 186.5800018310547| 182.5800018310547| 183.4199981689453| 177.7770538330078|4153700|   MMM|
|2021-01-29 00:00:00| 182.2899932861328|182.97999572753906| 174.869995117187

In [ ]:
# Lấy DATE - CLOSE - SYMBOL
#column_drop = ['_c0','Open','High','Low','Adj Close']
#dataSource = dataSource.drop(*column_drop)
df = dataSource.select(["Date","Close","Symbol"])

In [ ]:
dataSource.show()

In [ ]:
#Sắp xếp lại cột
current_data = dataSource.select("Symbol","Date","Close","Volume")
current_data.show()

In [ ]:
# LẤY NGÀY HIỆN TẠI VÀ N NGÀY TRƯỚC

In [ ]:
from datetime import date
from datetime import datetime, timedelta

DAYS_AGO_1 = 100
DAYS_AGO_7 = 7
DAYS_AGO_30 = 30

today = date.today()    
days_ago_1 = today - timedelta(days=DAYS_AGO_1)
days_ago_7 = today - timedelta(days=DAYS_AGO_7)
days_ago_30 = today - timedelta(days=DAYS_AGO_30)
print(today,days_ago_1,days_ago_7,days_ago_30)

In [ ]:
# Tạo 1 DF để lưu data
data_today = dataSource.select("Symbol","Date")
data_1ago = dataSource.select("Symbol","Date")
data_7ago = dataSource.select("Symbol","Date")
data_30ago = dataSource.select("Symbol","Date")

In [ ]:
# Lấy ngày cũ để test
today_temp = '2021-12-01'
ago1_temp = '2021-11-30'
ago7_temp = '2021-11-24'
ago30_temp = '2021-11-02'

In [ ]:
# Lọc Giá, volume theo ngày cho từng mã
data_today = current_data.filter(current_data.Date == today_temp)
data_1ago = current_data.filter(current_data.Date == ago1_temp)
data_7ago = current_data.filter(current_data.Date == ago7_temp)
data_30ago = current_data.filter(current_data.Date == ago30_temp)

In [ ]:
# Đổi tên dữ liệu cũ để phân biệt theo khung thời gian
data_1ago = data_1ago.withColumnRenamed('Close','Close_1D')
data_1ago = data_1ago.withColumnRenamed('Volume','Volume_1D')
data_7ago = data_7ago.withColumnRenamed('Close','Close_7D')
data_7ago = data_7ago.withColumnRenamed('Volume','Volume_7D')
data_30ago = data_30ago.withColumnRenamed('Close','Close_30D')
data_30ago = data_30ago.withColumnRenamed('Volume','Volume_30D')

In [ ]:
data_today.show(5)
data_1ago.show(5)
data_7ago.show(5)
data_30ago.show(5)

In [ ]:
#Xoá những cột không sử dụng
data_today = data_today.drop('Date')
data_1ago = data_1ago.drop('Symbol','Date')
data_7ago = data_7ago.drop('Symbol','Date')
data_30ago = data_30ago.drop('Symbol','Date')

In [ ]:
# Kết hợp các dataframe tạo thành tập dữ liệu chính , mọi QUERY đều thực hiện từ tập này
from pyspark.sql import *
from pyspark.sql.functions import *

w=Window.orderBy(lit(1))
#####
new_df=data_today.withColumn("rn",row_number().over(w)-1)
df_1ago=data_1ago.withColumn("rn",row_number().over(w)-1)

new_df = new_df.join(df_1ago,["rn"])
#####
df_7ago=data_7ago.withColumn("rn",row_number().over(w)-1)

new_df = new_df.join(df_7ago,["rn"])
#####
df_30ago=data_30ago.withColumn("rn",row_number().over(w)-1)

new_df = new_df.join(df_30ago,["rn"]).drop("rn")

In [ ]:
new_df.show()

In [ ]:
# Tính biến động theo khung 1d,7d,30d

new_df = new_df.withColumn("P_Change_1D",col("Close")/col("Close_1D")*100 - 100)
new_df = new_df.withColumn("P_Change_7D",col("Close")/col("Close_7D")*100 - 100)
new_df = new_df.withColumn("P_Change_30D",col("Close")/col("Close_30D")*100 - 100)

In [ ]:
## XOÁ CỘT THỪA SAU KHI TÍNH
columns = ['Close','Volume','Close_1D','Volume_1D','Close_7D','Volume_7D','Close_30D','Volume_30D']
P_Change_DF = new_df.drop(*columns)

In [ ]:
P_Change_DF.show()

In [ ]:
top5_giam_1d = P_Change_DF.sort('P_Change_1D').limit(5)
top5_giam_7d = P_Change_DF.sort('P_Change_7D').limit(5)
top5_giam_30d = P_Change_DF.sort('P_Change_30D').limit(5)

In [ ]:
top5_giam_1d.show()
top5_giam_7d.show()
top5_giam_30d.show()

In [ ]:
# Loại bỏ những cột không sử dụng trong từng df
top5_giam_1d = top5_giam_1d.drop('P_Change_7D','P_Change_30D')
top5_giam_7d = top5_giam_7d.drop('P_Change_1D','P_Change_30D')
top5_giam_30d = top5_giam_30d.drop('P_Change_1D','P_Change_7D')

In [ ]:
top5_giam_1d.show()
top5_giam_7d.show()
top5_giam_30d.show()

In [ ]:
df_ticker_giam = top5_giam_1d.withColumn("rn",row_number().over(w)-1)
df_7d=top5_giam_7d.withColumn("rn",row_number().over(w)-1)

df_ticker_giam = df_ticker_giam.join(df_7d,["rn"])
#####
df_30d=top5_giam_30d.withColumn("rn",row_number().over(w)-1)

df_ticker_giam = df_ticker_giam.join(df_30d,["rn"]).drop("rn")
#####

In [ ]:
df_ticker_giam.show()

In [ ]:
##### Top ticker tăng

In [ ]:
new_df = new_df.withColumn('P_Change_1D',col('P_Change_1D')*(-1))
new_df.show()

In [ ]:
new_df.sort('P_Change_1D').limit(3).show()

In [ ]:
# Top ticker có volume lớn

In [ ]:
df_big_volume = new_df.select('Symbol','Volume','Volume_1D','Volume_7D','Volume_30D')

In [ ]:
df_big_volume.show(20)

In [ ]:
# Ghi dữ liệu ra txt

import pandas as pd
import numpy as np

a_dataframe = pd.DataFrame([[1, 2, 3],[4, 5, 6],[7, 8, 9]])

numpy_array = a_dataframe.to_numpy()
np.savetxt("test_file.txt", numpy_array, fmt = "%d")

In [ ]:
type(a_dataframe)